In [ ]:
import torch
import tiktoken
import transformers
import datasets
import re

In [4]:
text1 = "The mother woke the child."
text2 = "The child woke his mother."

In [25]:
with open("exp_text.txt", "r") as f:
  text = f.read()
  
text

'Artificial intelligence (AI) is transforming the world at an unprecedented pace. From self-driving cars to personalized healthcare recommendations, AI-powered technologies are reshaping industries and redefining how we live and work. In education, AI tools help identify student learning patterns and customize content accordingly. In finance, algorithms detect fraudulent transactions in real-time. AI is also being used in agriculture to monitor crop health and optimize irrigation. However, with great power comes great responsibility. Concerns around data privacy, bias in decision-making, and job displacement are growing. To ensure ethical use, it’s essential for governments, organizations, and developers to collaborate on transparent policies. As AI continues to evolve, a balanced approach will help society maximize benefits while minimizing risks.'

In [ ]:
# oluşturulan tokenizer'ı kullanarak metinleri token'lara ayırma
from tokenizer import Tokenizer
create_tokenizer = Tokenizer("exp_text.json")
print(create_tokenizer.encode("minimizing."))
print(create_tokenizer.decode([100, 0]))
#vocab json dosyasında minimizing kelimesi var ancak bunu text'te minimiz ing olarak yazdığımıda vocab'ta karşığı olmamasına rağmen kombinasyon ile oluşturabiliyor.


[100, 0]
minimizing<unk>


In [26]:
create_tokens = create_tokenizer.encode(text)
print(create_tokens)

[2, 1, 3, 1, 4, 1, 5, 1, 6, 1, 7, 1, 8, 1, 9, 1, 10, 1, 11, 1, 12, 1, 13, 1, 14, 1, 15, 1, 16, 1, 17, 1, 18, 1, 19, 1, 20, 1, 21, 1, 22, 1, 23, 1, 24, 1, 25, 1, 26, 1, 27, 1, 28, 1, 29, 1, 25, 1, 30, 1, 31, 1, 32, 1, 33, 1, 34, 1, 35, 1, 36, 1, 37, 1, 38, 1, 39, 1, 25, 1, 40, 1, 41, 1, 42, 1, 31, 1, 43, 1, 44, 1, 45, 1, 46, 1, 47, 1, 48, 1, 49, 1, 33, 1, 5, 1, 50, 1, 51, 1, 52, 1, 48, 1, 53, 1, 16, 1, 54, 1, 55, 1, 56, 1, 25, 1, 57, 1, 58, 1, 59, 1, 60, 1, 61, 1, 62, 1, 63, 1, 61, 1, 64, 1, 65, 1, 66, 1, 67, 1, 68, 1, 69, 1, 48, 1, 70, 1, 25, 1, 71, 1, 72, 1, 22, 1, 73, 1, 74, 1, 75, 1, 76, 1, 77, 1, 78, 1, 79, 1, 80, 1, 81, 1, 82, 1, 25, 1, 83, 1, 16, 1, 84, 1, 85, 1, 86, 1, 87, 1, 88, 1, 33, 1, 89, 1, 16, 1, 90, 1, 91, 1, 92, 1, 93, 1, 94, 1, 35, 1, 96, 1, 97, 1, 98, 1, 99, 1, 100, 1, 101]


#### basic bir model eğitmek istersek: 

In [27]:
import sentencepiece as spm # google'ın geliştirdiği bir tokenizer kütüphanesi
spm.SentencePieceTrainer.Train(
    input='exp_text.txt', 
    model_prefix='spm_model', # model dosyasının ismi
    vocab_size=200, # kaç token olacağını belirliyor
    model_type='bpe' # bpe, unigram, char, word (bpe: byte pair encoding)
)

In [ ]:
spm_tokenizer = spm.SentencePieceProcessor(model_file='spm_model.model')
print(spm_tokenizer.encode(text))
spm_tokens = spm_tokenizer.encode(text, out_type=str)
print(spm_tokens)


[22, 169, 81, 77, 12, 33, 167, 28, 64, 157, 13, 126, 99, 192, 86, 97, 155, 176, 10, 9, 109, 91, 114, 4, 167, 4, 165, 129, 118, 5, 73, 25, 38, 23, 130, 181, 162, 100, 39, 54, 149, 189, 173, 80, 10, 15, 134, 35, 23, 76, 6, 12, 20, 38, 98, 172, 72, 34, 143, 176, 150, 95, 180, 44, 189, 175, 30, 151, 173, 9, 60, 111, 29, 168, 108, 18, 90, 34, 122, 103, 10, 33, 173, 83, 21, 18, 26, 34, 145, 3, 10, 32, 30, 24, 163, 162, 64, 71, 26, 91, 190, 181, 84, 162, 38, 125, 45, 180, 44, 35, 29, 170, 94, 43, 146, 81, 188, 54, 69, 173, 25, 162, 115, 133, 10, 23, 137, 17, 65, 26, 15, 83, 39, 55, 93, 25, 4, 107, 11, 173, 10, 116, 181, 84, 52, 3, 16, 13, 180, 85, 158, 78, 117, 31, 47, 148, 52, 169, 138, 70, 25, 97, 46, 36, 170, 33, 34, 12, 189, 82, 163, 181, 44, 86, 85, 123, 41, 163, 10, 87, 38, 33, 4, 159, 144, 69, 5, 35, 53, 6, 19, 11, 15, 169, 50, 98, 26, 162, 50, 82, 55, 43, 169, 21, 174, 45, 181, 128, 30, 48, 17, 180, 24, 78, 96, 23, 30, 17, 15, 39, 18, 96, 34, 51, 6, 67, 139, 19, 188, 181, 127, 6, 142,

In [31]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

In [38]:
hf_tokenizer = Tokenizer(BPE()) # BPE modelini kullanarak bir tokenizer oluştur
hf_tokenizer.pre_tokenizer = Whitespace() # boşluklara göre token'lara ayırma işlemi yapacak
trainer = BpeTrainer(vocab_size=8, special_tokens=["<unk>"]) # vocab boyutu ve özel token'lar
hf_tokenizer.train(["exp_text.txt"], trainer) # eğitimi başlat
hf_tokenizer.save("hf_tokenizer.json") # eğitilen tokenizer'ı kaydet,

In [39]:
print(hf_tokenizer.get_vocab_size())
print(hf_tokenizer.encode(text).ids)

38
[6, 28, 30, 20, 17, 20, 14, 20, 12, 23, 20, 25, 30, 16, 23, 23, 20, 18, 16, 25, 14, 16, 1, 6, 10, 2, 20, 29, 30, 28, 12, 25, 29, 17, 26, 28, 24, 20, 25, 18, 30, 19, 16, 33, 26, 28, 23, 15, 12, 30, 12, 25, 31, 25, 27, 28, 16, 14, 16, 15, 16, 25, 30, 16, 15, 27, 12, 14, 16, 5, 8, 28, 26, 24, 29, 16, 23, 17, 4, 15, 28, 20, 32, 20, 25, 18, 14, 12, 28, 29, 30, 26, 27, 16, 28, 29, 26, 25, 12, 23, 20, 36, 16, 15, 19, 16, 12, 23, 30, 19, 14, 12, 28, 16, 28, 16, 14, 26, 24, 24, 16, 25, 15, 12, 30, 20, 26, 25, 29, 3, 6, 10, 4, 27, 26, 33, 16, 28, 16, 15, 30, 16, 14, 19, 25, 26, 23, 26, 18, 20, 16, 29, 12, 28, 16, 28, 16, 29, 19, 12, 27, 20, 25, 18, 20, 25, 15, 31, 29, 30, 28, 20, 16, 29, 12, 25, 15, 28, 16, 15, 16, 17, 20, 25, 20, 25, 18, 19, 26, 33, 33, 16, 23, 20, 32, 16, 12, 25, 15, 33, 26, 28, 22, 5, 10, 25, 16, 15, 31, 14, 12, 30, 20, 26, 25, 3, 6, 10, 30, 26, 26, 23, 29, 19, 16, 23, 27, 20, 15, 16, 25, 30, 20, 17, 35, 29, 30, 31, 15, 16, 25, 30, 23, 16, 12, 28, 25, 20, 25, 18, 27, 12, 3

In [41]:
# oluşturulan tokenizer'ı kullanarak metinleri token'lara ayırma
from transformers import PreTrainedTokenizerFast
hf_pretrained_tokenizer = PreTrainedTokenizerFast(tokenizer_file="hf_tokenizer.json")
print(hf_pretrained_tokenizer.encode(text))
print(hf_pretrained_tokenizer.encode(text1))

[6, 28, 30, 20, 17, 20, 14, 20, 12, 23, 20, 25, 30, 16, 23, 23, 20, 18, 16, 25, 14, 16, 1, 6, 10, 2, 20, 29, 30, 28, 12, 25, 29, 17, 26, 28, 24, 20, 25, 18, 30, 19, 16, 33, 26, 28, 23, 15, 12, 30, 12, 25, 31, 25, 27, 28, 16, 14, 16, 15, 16, 25, 30, 16, 15, 27, 12, 14, 16, 5, 8, 28, 26, 24, 29, 16, 23, 17, 4, 15, 28, 20, 32, 20, 25, 18, 14, 12, 28, 29, 30, 26, 27, 16, 28, 29, 26, 25, 12, 23, 20, 36, 16, 15, 19, 16, 12, 23, 30, 19, 14, 12, 28, 16, 28, 16, 14, 26, 24, 24, 16, 25, 15, 12, 30, 20, 26, 25, 29, 3, 6, 10, 4, 27, 26, 33, 16, 28, 16, 15, 30, 16, 14, 19, 25, 26, 23, 26, 18, 20, 16, 29, 12, 28, 16, 28, 16, 29, 19, 12, 27, 20, 25, 18, 20, 25, 15, 31, 29, 30, 28, 20, 16, 29, 12, 25, 15, 28, 16, 15, 16, 17, 20, 25, 20, 25, 18, 19, 26, 33, 33, 16, 23, 20, 32, 16, 12, 25, 15, 33, 26, 28, 22, 5, 10, 25, 16, 15, 31, 14, 12, 30, 20, 26, 25, 3, 6, 10, 30, 26, 26, 23, 29, 19, 16, 23, 27, 20, 15, 16, 25, 30, 20, 17, 35, 29, 30, 31, 15, 16, 25, 30, 23, 16, 12, 28, 25, 20, 25, 18, 27, 12, 30, 